# 量的データ用（For quantitaive data）
[Methods]  
- Clustering methods : Kmeans, GMM, FSSEM, FSS-Kmeans    
- Allocation methods : random, proportional allocation, post stratification, optimal allocation

### 実験設定

In [1]:
clusters = 3  # クラスタ数
n_features_to_select = 17  # 選択される最大の特徴量数

N_SAMPLES = 100  # 標本サイズ
data_size = 3000 #データサイズ（df2)
H = clusters
N_TRIALS = 1000  # 試行回数
m_VALUE = 2  # 各クラスタの最小標本数(最適標本配分)
RANDOM_STATE = 0  # 乱数シード
ALLOCATION_LIST = ["random", "proportional", "post", "optimal"]
CRITERION_LIST = ["ml", "none"]
CLUSTERING_METHOD_LIST = ["kmeans"] #["gmm", "kmeans"]

-------------
### ライブラリのインポート＆その他の設定（Importing Libraries & Other Settings）

In [2]:
# 基本的なライブラリ
import numpy as np
import pandas as pd
from numpy.typing import NDArray
from scipy import stats

# Scikit-learn関連
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.utils.validation import check_X_y

# 抽象基底クラス (ABC)
from abc import ABCMeta, abstractmethod

# タイピングのサポート
from typing import Optional

# シード設定
from lightning.pytorch import seed_everything

seed_everything(8)

# 可視化の設定
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.style.use("ggplot")

Seed set to 8


-----------
### データの前処理（Data Preprocessing）

##### 外れ値を除去する関数

In [3]:
# 外れ値の除去
def remove_outliers_zscore(
    data: pd.DataFrame, metric: str, threshold: float = 2
) -> pd.DataFrame:
    z_scores = np.abs(stats.zscore(data[metric]))
    data = data[(z_scores < threshold)]
    return data

##### メルカリデータ（df1：全部, df2：一部）

In [4]:
# df1(all the data)
df1 = pd.read_csv(
    R"C:\Users\HaruMomozu\Documents\aug_first_cpn_data_for_ab_test_sensibility_tsukuba.csv"
)
obj1 = "GMV"
features_list1 = [
    "hist_4_day_buy_num",
    "hist_4_day_gmv",
    "his_4_day_is_buy",
    "hist_30_day_buy_days",
    "hist_30_day_buy_num",
    "hist_30_day_gmv",
    "hist_30_day_buy_recency",
    "hist_30_day_pay_days",
    "hist_30_day_atpu",
    "hist_30_day_gpv",
    "hist_30_day_pay_recency",
    "hist_30_day_list_days",
    "hist_30_day_list_num",
    "hist_30_day_list_recency",
    "hist_30_day_like_count",
    "hist_30_day_like_count_not_deleted",
    "hist_30_day_like_recency",
]

# df2(subset of the data)
df2 = df1.iloc[:data_size]
obj2 = obj1
features_list2 = features_list1

##### 外れ値除去と標準化

In [5]:
df = df2  # choose from (df1, df2)
obj = obj2  # choose from (obj1, obj2)
features_list = features_list2  # choose from (features_list1, features_list2)

# 外れ値除去
df = remove_outliers_zscore(df, obj)

# 標準化
X = df[features_list]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(X)
X_scaled = pd.DataFrame(scaled_features, columns=features_list)

# 目的変数
y = df[obj]

# 行を詰める
df = df.reset_index(drop=True)

-------------
### 提案手法のクラスタリング

##### Allocation_in_Wrapper Class

In [6]:
class Allocation_in_Wrapper(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        n_features_to_select: int,
        n_clusters: int,
        allocation_method_list: list[str] = ["random"],
        clustering_method: str = "gmm",
        random_state: int = 0,
    ):
        self.n_features_to_select = n_features_to_select  # 特徴量数
        self.n_clusters = n_clusters  # クラスタ数
        self.allocation_method_list = allocation_method_list  # 特徴量選択基準
        self.clustering_method = clustering_method  # クラスタリング手法
        self.random_state = random_state

    def fss(self, X: pd.DataFrame, y: pd.DataFrame) -> "Allocation_in_Wrapper":
        X, y = check_X_y(X, y)

        n_features = X.shape[1]  # 総特徴量数
        self.selected_features_ = []  # ここに選択した特徴量を入れる

        # 選ばれた特徴量と残っている特徴量の初期化
        current_features = []
        remaining_features = list(range(n_features))
        best_score = -np.inf

        while len(current_features) < self.n_features_to_select:
            best_feature = None  # 選ぶ特徴量の初期化

            for feature in remaining_features:
                temp_features = tuple(
                    current_features + [feature]
                )  # 特徴量をひとつ加え、score計算

                score = self.crit(X[:, temp_features])

                if score > best_score:
                    best_score = score
                    best_feature = feature

            if best_feature is not None:
                current_features.append(
                    best_feature
                )  # best feature をcurrent features に追加
                remaining_features.remove(
                    best_feature
                )  # best feature をremaining features から取り除く
                self.selected_features_ = current_features
            else:
                break

        # 選ばれた特徴量サブセットでクラスタリング
        final_features = X[:, self.selected_features_]
        if self.clustering_method == "gmm":
            self.final_model_ = GaussianMixture(
                n_components=self.n_clusters,
                random_state=self.random_state,
                init_params="kmeans",
            )
        elif self.clustering_method == "kmeans":
            self.final_model_ = KMeans(
                n_clusters=self.n_clusters, random_state=self.random_state
            )
        else:
            raise ValueError(f"Unknown clustering method: {self.clustering_method}")

        self.final_model_.fit(final_features)
        self.final_cluster_assignments_ = self.final_model_.predict(final_features)

        return self

    def crit(self, X: pd.DataFrame) -> float:
        # クラスタリング手法がGMMの場合
        if self.clustering_method == "gmm":
            em = GaussianMixture(
                n_components=self.n_clusters,
                random_state=self.random_state,
                init_params="kmeans",
            )
            em.fit(X)
            self.cluster_label = em.predict(X)
            self.cluster_size = np.bincount(self.cluster_label, minlength=clusters)

        # クラスタリング手法がKMEANSの場合
        if self.clustering_method == "kmeans":
            kmeans = KMeans(
                n_clusters=self.n_clusters,
                random_state=self.random_state,
            )
            kmeans.fit(X)
            self.cluster_label = kmeans.predict(X)
            self.cluster_size = np.bincount(self.cluster_label, minlength=clusters)

        



        score = 5

        return score
    

    # 各ポリシーの生成を行う関数
    def create_policies(
        outer_self,
        n_samples: int,
        H: int,
        random_state: int,
        m_value: int,
    ) -> dict[list[str] : list[BaseAllocation]]:
        policies_dict = {}
        if "random" in outer_self.allocation_method_list:
            policies_dict["random"]=outer_self.RandomAllocation(
                n_samples=n_samples,
                H=H,
                random_state=random_state,
            )
        if "proportional" in outer_self.allocation_method_list:
            policies_dict["proportional"]=outer_self.ProportionalAllocation(
                n_samples=n_samples,
                H=H,
                random_state=random_state,
            )
        if "post" in outer_self.allocation_method_list:
            policies_dict["post"] = outer_self.PostStratification(
                n_samples=n_samples,
                H=H,
                random_state=random_state,
            )
        if "optimal" in outer_self.allocation_method_list:
            policies_dict["optimal"] = outer_self.OptimalAllocation(
                n_samples=n_samples,
                H=H,
                random_state=random_state,
                m=np.full(H, m_value),
                M=None,
            )
        return policies_dict

    class BaseAllocation(metaclass=ABCMeta):  # 抽象基底クラス（ABC）
        # 初期化クラス（n_samples(標本サイズ), H(クラスタ数)）
        def __init__(
            self,
            outer_self,
            n_samples: int,
            H: int,
            random_state: int,
        ):
            self.outer_self = outer_self
            self.n_samples = n_samples
            self.H = H
            self.random_state = random_state

        @abstractmethod
        def solve(self, X: NDArray, y: NDArray) -> NDArray:
            """標本配分を解く

            Args:
                X (NDArray): データ (N x M)
                y (NDArray): 目的変数 (N)

            Raises:
                NotImplementedError: _description_

            Returns:
                NDArray: 各クラスタの標本数 (H, )

            Note:
                M: 特徴量数
                H: クラスタ数
            """
            # 具象クラスがsolveメゾッドを実装しない場合はNotImpleamentedErrorが発生
            raise NotImplementedError

        def clustering(self, X: NDArray) -> tuple[NDArray, NDArray]:
            cluster_label = self.outer_self.cluster_label
            cluster_size = self.outer_self.cluster_size
            # インスタンス変数として設定
            self.cluster_label = cluster_label
            self.N = cluster_size
            return cluster_label, cluster_size
    
    class RandomAllocation(BaseAllocation):
        # 抽象メゾッドを具象化
        def solve(self, X: NDArray, y: NDArray) -> NDArray:
            """ランダムにn_samplesの標本を選択する"""
            n = np.array([self.n_samples])
            return n

        def clustering(self, X: NDArray) -> tuple[NDArray, NDArray]:
            # cluster_labelのすべての要素は0（すべてのデータを同じクラスタに属させている）
            cluster_label = np.zeros(
                X.shape[0]
            )  # cluster_label = [0,0,0,,...(要素数：データ数）]
            # クラスタサイズ＝データ数
            cluster_size = np.array([len(cluster_label)])  # cluster_size=[データ数]
            return cluster_label, cluster_size
        
    class ProportionalAllocation(BaseAllocation):
        def solve(self, X: NDArray, y: NDArray) -> NDArray:
            """各クラスタ数に比例した標本数で分割する"""
            n: NDArray = np.round(self.N / self.N.sum() * self.n_samples).astype(int)

            if n.sum() > self.n_samples:
                # nの合計がn_samplesより大きい場合は一番標本数が多いクラスタから削る
                n[np.argmax(n)] -= n.sum() - self.n_samples

            for i in range(
                len(n)
            ):  # nの要素でm_VALUEより小さいものがあれば要素数が最も大きい層から持ってくる
                if n[i] < m_VALUE:
                    delta = m_VALUE - n[i]
                    n[i] = m_VALUE
                    n[np.argmax(n)] -= delta

            return n

    class PostStratification(BaseAllocation):
        def solve(self, X: NDArray, y: NDArray) -> NDArray:
            """ランダムにn_samplesの標本を選択する"""
            n = np.array([self.n_samples])

            return n  # （例）n=[標本サイズ]
        
    class OptimalAllocation(BaseAllocation):
        def __init__(
            self,
            n_samples: int,
            H: int,
            m: NDArray,  # 標本サイズ下限
            M: Optional[NDArray] = None,  # 標本サイズ上限 #Optional(Noneである可能性がある)
            random_state: int = 0,

        ):
            super().__init__(
                n_samples, H, random_state
            )  # 基底クラスBaseAllocation（スーパークラス）の初期化メゾッドを呼び出す
            self.m = m  # 各クラスタの最小標本サイズ (H, )
            self.M = M  # 各クラスタの最大標本サイズ (H, ), (指定しない場合はクラスタサイズ)

        def solve(self, X: NDArray, y: NDArray) -> NDArray:
            # S:クラスタ毎の目的変数のvarianceを要素とする配列 (H, )
            S = np.array([np.var(y[self.cluster_label == h]) for h in range(self.H)])
            d = (self.N**2) * S  # (H, )
            n = self._simple_greedy(n=self.m.copy(), d=d)

            # 制約チェック
            self._check_constraints(n)

            return n

        def _simple_greedy(self, n: NDArray, d: NDArray) -> NDArray:
            print(n)
            print(d)
            M = self.M.copy() if self.M is not None else self.N.copy()
            I = np.arange(self.H)  # noqa #クラスタのインデックス
            while (n.sum() != self.n_samples) and len(I) != 0:
                delta = np.zeros(self.H)
                delta[I] = (d / (n + 1) - d / n)[I]
                h_star = np.argmin(delta[I])
                h_star = I[h_star]

                if n[h_star] + 1 <= M[h_star]:
                    n[h_star] = n[h_star] + 1
                else:
                    # Iの要素h_starを削除
                    I_ = I.tolist()
                    I_ = [i for i in I_ if i != h_star]
                    I = np.array(I_)  # noqa

            return n

        def _check_constraints(self, n: NDArray):
            assert (
                n.sum() <= self.n_samples
            ), f"Total sample size is over than {self.n_samples}"
            assert np.all(n >= self.m), "Minimum sample size constraint is not satisfied"
            if self.M is not None:
                assert np.all(
                    n <= self.M
                ), "Maximum sample size constraint is not satisfied"

    def estimate_y_mean(n: NDArray, cluster_label: NDArray, y: NDArray) -> NDArray:
        """実際にサンプリングを行って目的変数の平均を推定

        Args:
            n (NDArray): 各クラスタの標本数 (H, )
            cluster_label (NDArray): クラスタラベル (N, )
            y (NDArray): 目的変数 (N, )

        Returns:
            NDArray: 推定された目的変数の平均

        Note:
            N: データ数
            H: クラスタ数
        """
        # cluster_labelからユニークなクラスタラベルを取得し、母集団の各クラスタのサイズNを取得
        N = np.unique(cluster_label, return_counts=True)[1]  # クラスタサイズ (H, )
        weights = N / N.sum()
        y_hat = 0
        for h in range(n.shape[0]):  # n.shape[0]:層の数
            y_cluster = y[cluster_label == h]
            # クラスタ内でランダム n_h サンプリング
            sample: NDArray = np.random.choice(y_cluster, n[h], replace=False)
            y_sample_mean = sample.mean()  # サンプリングした標本の平均
            y_hat += y_sample_mean * weights[h]

        return y_hat


    def estimate_y_mean_post(n: NDArray, cluster_label: NDArray, y: NDArray) -> NDArray:
        N = np.unique(cluster_label, return_counts=True)[1]
        weights = N / N.sum()
        y_hat = 0
        indices = np.arange(N.sum())
        y_array = np.array(y.tolist())
        n_indices = np.random.choice(indices, n[0], replace=False)
        n_label = np.array([cluster_label[i] for i in n_indices])
        n_new = np.unique(n_label)
        for h in n_new:
            index = np.where(n_label == h)[0]
            sample = y_array[n_indices[index]]
            y_sample_mean = sample.mean()  # サンプリングした標本の平均
            y_hat += y_sample_mean * weights[h]
        return y_hat             
    

    def get_feature_index_out(self) -> NDArray:
        return np.array(self.selected_features_)  # 選択された特徴量のインデックス

    def get_final_cluster_assignments(self) -> NDArray:
        return self.final_cluster_assignments_  # 最終的なクラスタリング結果

##### Wrapper classでクラスタリングしたとき

In [7]:
# Wrapper法を実装するための関数
def process_wrapper(
    name: str, instance: "Allocation_in_Wrapper", X_scaled: pd.DataFrame, y: pd.DataFrame
):
    instance.fss(X_scaled, y)
    selected_features_index = instance.get_feature_index_out()
    cluster_label = instance.get_final_cluster_assignments()
    cluster_size = np.unique(cluster_label, return_counts=True)[1]

    return selected_features_index, cluster_label, cluster_size


# Wrapperインスタンスのリスト
instances = [
    (
        "fssem_tr",
        Wrapper(
            n_features_to_select=n_features_to_select,
            n_clusters=clusters,
            criterion="tr",
            clustering_method="gmm",
            random_state=0,
        ),
    ),
    (
        "fssem_ml",
        Wrapper(
            n_features_to_select=n_features_to_select,
            n_clusters=clusters,
            criterion="ml",
            clustering_method="gmm",
            random_state=0,
        ),
    ),
    (
        "fsskmeans_tr",
        Wrapper(
            n_features_to_select=n_features_to_select,
            n_clusters=clusters,
            criterion="tr",
            clustering_method="kmeans",
            random_state=0,
        ),
    ),
    (
        "fsskmeans_ml",
        Wrapper(
            n_features_to_select=n_features_to_select,
            n_clusters=clusters,
            criterion="ml",
            clustering_method="kmeans",
            random_state=0,
        ),
    ),
]

# 辞書の初期化
selected_features_index_dict = {}
cluster_label_dict = {}
cluster_size_dict = {}

# 各インスタンスに対して処理を実行
for name, instance in instances:
    selected_features_index, cluster_label, cluster_size = process_wrapper(
        name, instance, X_scaled, y
    )
    selected_features_index_dict[name] = selected_features_index
    cluster_label_dict[name] = cluster_label
    cluster_size_dict[name] = cluster_size
    print("[", name, "]")
    print("選択された特徴量のインデックス : ", selected_features_index)
    print("選択された特徴量の数 : ", len(selected_features_index))
    print("各層のクラスタサイズ : ", cluster_size)
    print()


c:\Users\HaruMomozu\Desktop\momozu\ABtesting\.venv\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


[ fssem_tr ]
選択された特徴量のインデックス :  [ 2  6  0  4 13 11  7 10  3 16  8  1]
選択された特徴量の数 :  12
各層のクラスタサイズ :  [1529 1431]

[ fssem_ml ]
選択された特徴量のインデックス :  [2 0 1]
選択された特徴量の数 :  3
各層のクラスタサイズ :  [1529 1431]

[ fsskmeans_tr ]
選択された特徴量のインデックス :  [ 2  6 14 13  7 10  1  8  9]
選択された特徴量の数 :  9
各層のクラスタサイズ :  [1529 1431]

[ fsskmeans_ml ]
選択された特徴量のインデックス :  [ 5 14  9 15  7 12  2  1 10 16  3  4 13  8  6 11  0]
選択された特徴量の数 :  17
各層のクラスタサイズ :  [ 810 2150]



##### Kmeans, GMMでクラスタリングしたとき

In [8]:
# k-meansクラスタリングの適用
kmeans = KMeans(n_clusters=clusters, random_state=0)
kmeans_cluster = kmeans.fit_predict(X_scaled)

gmm = GaussianMixture(n_components=clusters, random_state=0, init_params="kmeans")
gmm_cluster = gmm.fit_predict(X_scaled)

kmeans_size = np.bincount(kmeans_cluster, minlength=clusters)
gmm_size = np.bincount(gmm_cluster, minlength=clusters)

cluster_label_dict["kmeans"] = kmeans_cluster
cluster_label_dict["gmm"] = gmm_cluster
cluster_size_dict["kmeans"] = kmeans_size
cluster_size_dict["gmm"] = gmm_size

methods = ["kmeans", "gmm"]
for name in methods:
    print("[", name, "]")
    print("各層のクラスタサイズ : ", cluster_size_dict[name])
    print()


[ kmeans ]
各層のクラスタサイズ :  [ 810 2150]

[ gmm ]
各層のクラスタサイズ :  [ 766 2194]



-----------------------------
### 標本配分（Allocation)

##### 基底クラス

In [9]:
class BaseAllocation(metaclass=ABCMeta):  # 抽象基底クラス（ABC）
    # 初期化クラス（n_samples(標本サイズ), H(クラスタ数)）
    def __init__(
        self,
        n_samples: int,
        H: int,
        random_state: int,
        criterion: str,
        clustering_method: str,
    ):
        self.n_samples = n_samples
        self.H = H
        self.random_state = random_state
        self.criterion = criterion
        self.clustering_method = clustering_method

    @abstractmethod
    def solve(self, X: NDArray, y: NDArray) -> NDArray:
        """標本配分を解く

        Args:
            X (NDArray): データ (N x M)
            y (NDArray): 目的変数 (N)

        Raises:
            NotImplementedError: _description_

        Returns:
            NDArray: 各クラスタの標本数 (H, )

        Note:
            M: 特徴量数
            H: クラスタ数
        """
        # 具象クラスがsolveメゾッドを実装しない場合はNotImpleamentedErrorが発生
        raise NotImplementedError

    def clustering(self, X: NDArray) -> tuple[NDArray, NDArray]:
        if self.criterion == "tr" and self.clustering_method == "gmm":
            cluster_label = cluster_label_dict["fssem_tr"]
            cluster_size = cluster_size_dict["fssem_tr"]
        if self.criterion == "ml" and self.clustering_method == "gmm":
            cluster_label = cluster_label_dict["fssem_ml"]
            cluster_size = cluster_size_dict["fssem_ml"]
        if self.criterion == "tr" and self.clustering_method == "kmeans":
            cluster_label = cluster_label_dict["fsskmeans_tr"]
            cluster_size = cluster_size_dict["fsskmeans_tr"]
        if self.criterion == "ml" and self.clustering_method == "kmeans":
            cluster_label = cluster_label_dict["fsskmeans_ml"]
            cluster_size = cluster_size_dict["fsskmeans_ml"]
        if self.criterion == "none" and self.clustering_method == "kmeans":
            cluster_label = cluster_label_dict["kmeans"]
            cluster_size = cluster_size_dict["kmeans"]
        if self.criterion == "none" and self.clustering_method == "gmm":
            cluster_label = cluster_label_dict["gmm"]
            cluster_size = cluster_size_dict["gmm"]

        # インスタンス変数として設定
        self.cluster_label = cluster_label
        self.N = cluster_size
        return cluster_label, cluster_size

##### 単純無作為抽出のクラス

In [10]:
class RandomAllocation(BaseAllocation):
    # 抽象メゾッドを具象化
    def solve(self, X: NDArray, y: NDArray) -> NDArray:
        """ランダムにn_samplesの標本を選択する"""
        n = np.array([self.n_samples])
        return n

    def clustering(self, X: NDArray) -> tuple[NDArray, NDArray]:
        # cluster_labelのすべての要素は0（すべてのデータを同じクラスタに属させている）
        cluster_label = np.zeros(
            X.shape[0]
        )  # cluster_label = [0,0,0,,...(要素数：データ数）]
        # クラスタサイズ＝データ数
        cluster_size = np.array([len(cluster_label)])  # cluster_size=[データ数]
        return cluster_label, cluster_size

##### 比例配分のクラス

In [11]:
class ProportionalAllocation(BaseAllocation):
    def solve(self, X: NDArray, y: NDArray) -> NDArray:
        """各クラスタ数に比例した標本数で分割する"""
        n: NDArray = np.round(self.N / self.N.sum() * self.n_samples).astype(int)

        if n.sum() > self.n_samples:
            # nの合計がn_samplesより大きい場合は一番標本数が多いクラスタから削る
            n[np.argmax(n)] -= n.sum() - self.n_samples

        for i in range(
            len(n)
        ):  # nの要素でm_VALUEより小さいものがあれば要素数が最も大きい層から持ってくる
            if n[i] < m_VALUE:
                delta = m_VALUE - n[i]
                n[i] = m_VALUE
                n[np.argmax(n)] -= delta

        return n

##### 事後層化のクラス

In [12]:
class PostStratification(BaseAllocation):
    def solve(self, X: NDArray, y: NDArray) -> NDArray:
        """ランダムにn_samplesの標本を選択する"""
        n = np.array([self.n_samples])

        return n  # （例）n=[標本サイズ]

##### 最適標本配分のクラス

In [13]:
class OptimalAllocation(BaseAllocation):
    def __init__(
        self,
        n_samples: int,
        H: int,
        m: NDArray,  # 標本サイズ下限
        M: Optional[NDArray] = None,  # 標本サイズ上限 #Optional(Noneである可能性がある)
        random_state: int = 0,
        criterion: str = "tr",
        clustering_method: str = "kmeans",
    ):
        super().__init__(
            n_samples, H, random_state, criterion, clustering_method
        )  # 基底クラスBaseAllocation（スーパークラス）の初期化メゾッドを呼び出す
        self.m = m  # 各クラスタの最小標本サイズ (H, )
        self.M = M  # 各クラスタの最大標本サイズ (H, ), (指定しない場合はクラスタサイズ)

    def solve(self, X: NDArray, y: NDArray) -> NDArray:
        # S:クラスタ毎の目的変数のvarianceを要素とする配列 (H, )
        S = np.array([np.var(y[self.cluster_label == h]) for h in range(self.H)])
        d = (self.N**2) * S  # (H, )
        n = self._simple_greedy(n=self.m.copy(), d=d)

        # 制約チェック
        self._check_constraints(n)

        return n

    def _simple_greedy(self, n: NDArray, d: NDArray) -> NDArray:
        print(n)
        print(d)
        M = self.M.copy() if self.M is not None else self.N.copy()
        I = np.arange(self.H)  # noqa #クラスタのインデックス
        while (n.sum() != self.n_samples) and len(I) != 0:
            delta = np.zeros(self.H)
            delta[I] = (d / (n + 1) - d / n)[I]
            h_star = np.argmin(delta[I])
            h_star = I[h_star]

            if n[h_star] + 1 <= M[h_star]:
                n[h_star] = n[h_star] + 1
            else:
                # Iの要素h_starを削除
                I_ = I.tolist()
                I_ = [i for i in I_ if i != h_star]
                I = np.array(I_)  # noqa

        return n

    def _check_constraints(self, n: NDArray):
        assert (
            n.sum() <= self.n_samples
        ), f"Total sample size is over than {self.n_samples}"
        assert np.all(n >= self.m), "Minimum sample size constraint is not satisfied"
        if self.M is not None:
            assert np.all(
                n <= self.M
            ), "Maximum sample size constraint is not satisfied"

##### 母平均の推定値を計算する関数

In [14]:
def estimate_y_mean(n: NDArray, cluster_label: NDArray, y: NDArray) -> NDArray:
    """実際にサンプリングを行って目的変数の平均を推定

    Args:
        n (NDArray): 各クラスタの標本数 (H, )
        cluster_label (NDArray): クラスタラベル (N, )
        y (NDArray): 目的変数 (N, )

    Returns:
        NDArray: 推定された目的変数の平均

    Note:
        N: データ数
        H: クラスタ数
    """
    # cluster_labelからユニークなクラスタラベルを取得し、母集団の各クラスタのサイズNを取得
    N = np.unique(cluster_label, return_counts=True)[1]  # クラスタサイズ (H, )
    weights = N / N.sum()
    y_hat = 0
    for h in range(n.shape[0]):  # n.shape[0]:層の数
        y_cluster = y[cluster_label == h]
        # クラスタ内でランダム n_h サンプリング
        sample: NDArray = np.random.choice(y_cluster, n[h], replace=False)
        y_sample_mean = sample.mean()  # サンプリングした標本の平均
        y_hat += y_sample_mean * weights[h]

    return y_hat


def estimate_y_mean_post(n: NDArray, cluster_label: NDArray, y: NDArray) -> NDArray:
    N = np.unique(cluster_label, return_counts=True)[1]
    weights = N / N.sum()
    y_hat = 0
    indices = np.arange(N.sum())
    y_array = np.array(y.tolist())
    n_indices = np.random.choice(indices, n[0], replace=False)
    n_label = np.array([cluster_label[i] for i in n_indices])
    n_new = np.unique(n_label)
    for h in n_new:
        index = np.where(n_label == h)[0]
        sample = y_array[n_indices[index]]
        y_sample_mean = sample.mean()  # サンプリングした標本の平均
        y_hat += y_sample_mean * weights[h]
    return y_hat

##### 各ポリシーを生成

In [15]:
# 各ポリシーの生成を行う関数
def create_policies(
    criterion_list: list[str],
    clustering_method_list: list[str],
    n_samples: int,
    H: int,
    random_state: int,
    m_value: int,
) -> dict[list[str] : list[BaseAllocation]]:
    policies_dict = {}
    for criterion in criterion_list:
        for clustering_method in clustering_method_list:
            policies: list[BaseAllocation] = [
                RandomAllocation(
                    n_samples=n_samples,
                    H=H,
                    random_state=random_state,
                    criterion=criterion,
                    clustering_method=clustering_method,
                ),
                ProportionalAllocation(
                    n_samples=n_samples,
                    H=H,
                    random_state=random_state,
                    criterion=criterion,
                    clustering_method=clustering_method,
                ),
                PostStratification(
                    n_samples=n_samples,
                    H=H,
                    random_state=random_state,
                    criterion=criterion,
                    clustering_method=clustering_method,
                ),
                OptimalAllocation(
                    n_samples=n_samples,
                    H=H,
                    random_state=random_state,
                    m=np.full(H, m_value),
                    M=None,
                    criterion=criterion,
                    clustering_method=clustering_method,
                ),
            ]
            policies_dict[(criterion, clustering_method)] = policies
    return policies_dict

----------------
### 母平均の推定と分散の計算

In [16]:
##### 母平均の推定

In [17]:
policies_dict = create_policies(
    CRITERION_LIST, CLUSTERING_METHOD_LIST, N_SAMPLES, H, RANDOM_STATE, m_VALUE
)
allocations_dict = {}
for method, policies in policies_dict.items():
    # それぞれの戦略で各クラスタの標本数を求解
    allocations: list[dict] = []  # 各戦略の実行結果が辞書形式で追加される
    for policy in policies:
        # policyを用いてXをクラスタリング
        cluster_label, _ = policy.clustering(X_scaled)
        n = policy.solve(X_scaled, y)
        allocations.append(
            {
                "policy": policy.__class__.__name__,
                "n": n,
                "cluster_label": cluster_label,
            }
        )
    allocations_dict[method] = allocations

# 各戦略の標本数に基づいて目的変数の平均を推定
y_hats_dict = {}
for method, allocations in allocations_dict.items():
    y_hats = []
    for random_state in range(N_TRIALS):
        for allocation in allocations:
            if allocation["policy"] == "PostStratification":
                y_hat = estimate_y_mean_post(
                    allocation["n"], allocation["cluster_label"], y
                )
            else:
                y_hat = estimate_y_mean(allocation["n"], allocation["cluster_label"], y)
            y_hats.append(
                {
                    "policy": allocation["policy"],
                    "y_hat": y_hat,
                    "random_state": random_state,
                }
            )
    y_hats_dict[method] = y_hats

[2 2]
[5.61260709e+13 2.23524856e+13]
[2 2]
[2.24750549e+13 4.79357132e+13]
[2 2]
[5.61260709e+13 2.23524856e+13]
[2 2]
[5.61260709e+13 2.23524856e+13]
[2 2]
[1.46310681e+13 7.42317620e+13]
[2 2]
[2.24750549e+13 4.79357132e+13]


##### 結果の出力

In [18]:
for method, y_hats in y_hats_dict.items():
    y_hat_df = pd.DataFrame(y_hats)
    y_hat_df["error"] = (
        y_hat_df["y_hat"] - y.mean()
    )  # 真の平均からの誤差をerrorカラムに追加

    # random_allocationの誤差分散
    random_allocation_std = y_hat_df[y_hat_df["policy"] == "RandomAllocation"][
        "error"
    ].var()
    # random_allocation以外の誤差分散
    non_random_allocation_std = (
        y_hat_df[y_hat_df["policy"] != "RandomAllocation"]
        .groupby("policy")["error"]
        .var()
    )

    # 削減率
    reduction_rate = (1 - non_random_allocation_std / random_allocation_std) * 100

    ## policyの順番をpoliciesの順番に調整
    reduction_rate = reduction_rate.reindex(
        [policy.__class__.__name__ for policy in policies]
    )

    print("")
    print("(clustering_method:", method[1], ", criterion:", method[0], ")")
    print("[各手法の誤差分散削減率]")
    print(reduction_rate)

    print("[各手法の分散]")
    policy_name_list = [policy.__class__.__name__ for policy in policies]
    for i in range(len(policy_name_list)):
        var = y_hat_df[y_hat_df["policy"] == policy_name_list[i]]["error"].var()
        print(policy_name_list[i], var)


(clustering_method: gmm , criterion: ml )
[各手法の誤差分散削減率]
policy
RandomAllocation               NaN
ProportionalAllocation    6.247977
PostStratification        3.460032
OptimalAllocation         8.776575
Name: error, dtype: float64
[各手法の分散]
RandomAllocation 174605.50959078077
ProportionalAllocation 163696.19675981626
PostStratification 168564.10234339567
OptimalAllocation 159281.12677647927

(clustering_method: kmeans , criterion: ml )
[各手法の誤差分散削減率]
policy
RandomAllocation                NaN
ProportionalAllocation     3.376753
PostStratification        -9.727350
OptimalAllocation         10.446356
Name: error, dtype: float64
[各手法の分散]
RandomAllocation 161337.10977551303
ProportionalAllocation 155889.1537394893
PostStratification 177030.93570985756
OptimalAllocation 144483.2615712785

(clustering_method: gmm , criterion: tr )
[各手法の誤差分散削減率]
policy
RandomAllocation                NaN
ProportionalAllocation     1.405573
PostStratification         3.555973
OptimalAllocation         12.857874